# Tutorial 04: KJ Method Visualization

Goal: Visualize the result in Obsidian Canvas format.

We take the DocumentTree from the previous step and export it to a .canvas file.

In [ ]:
import json
import logging
from pathlib import Path
import uuid

from matome.exporters.obsidian import ObsidianCanvasExporter
from matome.utils.store import DiskChunkStore
from domain_models.manifest import DocumentTree, SummaryNode, Chunk

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

USE_MOCK = True

## 1. Load Tree Data
We load the tree structure generated in the previous tutorial.
If the file is missing (e.g., running standalone), we create a dummy tree.

In [ ]:
tree_file = Path("tree.json")
tree = None

if tree_file.exists():
    print("Loading tree from tree.json...")
    with open(tree_file, "r", encoding="utf-8") as f:
        tree = DocumentTree.model_validate_json(f.read())
else:
    print("tree.json not found. Creating dummy tree for demonstration.")
    # Create Dummy Tree
    root_id = str(uuid.uuid4())
    child1_id = str(uuid.uuid4())
    child2_id = str(uuid.uuid4())
    
    # Root
    root = SummaryNode(
        id=root_id, text="Root Summary: The main topic.", level=2, children_indices=[child1_id, child2_id]
    )
    
    # Children (Summaries of chunks)
    child1 = SummaryNode(
        id=child1_id, text="Summary 1: Section A", level=1, children_indices=[0, 1]
    )
    child2 = SummaryNode(
        id=child2_id, text="Summary 2: Section B", level=1, children_indices=[2]
    )
    
    all_nodes = {root_id: root, child1_id: child1, child2_id: child2}
    leaf_ids = [0, 1, 2]
    
    tree = DocumentTree(root_node=root, all_nodes=all_nodes, leaf_chunk_ids=leaf_ids)

## 2. Load Chunk Store (Optional)
To display full text of leaf chunks, we need access to the chunk store.

In [ ]:
db_path = Path("chunks.db")
store = None

if db_path.exists():
    print("Opening chunks.db...")
    store = DiskChunkStore(db_path=db_path)
else:
    print("chunks.db not found. Creating temporary store with dummy chunks.")
    store = DiskChunkStore()
    store.add_chunks([
        Chunk(index=0, text="Chunk 0 content.", start_char_idx=0, end_char_idx=10),
        Chunk(index=1, text="Chunk 1 content.", start_char_idx=10, end_char_idx=20),
        Chunk(index=2, text="Chunk 2 content.", start_char_idx=20, end_char_idx=30),
    ])

## 3. Export to Obsidian Canvas
We generate the .canvas file.

In [ ]:
exporter = ObsidianCanvasExporter()
output_path = Path("summary_kj.canvas")

try:
    exporter.export(tree, output_path, store)
    print(f"Exported to {output_path.absolute()}")
except Exception as e:
    print(f"Export failed: {e}")
    raise
finally:
    if store:
        store.close()

## 4. Verification
Check file content (first few lines).

In [ ]:
if output_path.exists():
    with open(output_path, "r", encoding="utf-8") as f:
        content = f.read(500)
        print(content + "...")